In [1]:
import numpy as np
import os
import glob

from PIL import Image

In [2]:
train_files = sorted(glob.glob('amharic_fidel_images/fidels/*/*.jpg'))
test_files = sorted(glob.glob('amharic_fidel_images/fidel_test/*/*.jpg'))

In [22]:
I = Image.open(train_files[0]).size
print('I : ',I)
j = []
k = []
for i in train_files:
    #print("the size of " , train_files.index(i), "\t" , Image.open(i).size) 
    if(Image.open(i).size == I):
        j.append(train_files.index(i))
    else:
        k.append(train_files.index(i))
        

# print(j)
print('k : ', k)
print(Image.open(train_files[225]).size)
J = Image.open(train_files[225]).size
print('J :', J)

I :  (720, 438)
k :  [225]
(720, 446)
J : (720, 446)


In [7]:
all_x = []
all_y = []

for i in train_files:
    if(i == train_files[225]):
        all_x.append(np.array(Image.open(i).convert('L')).reshape(-1, 1)[:315360, :])
    else:
        all_x.append(np.array(Image.open(i).convert('L')).reshape(-1, 1))
    all_y.append(i.split('/')[2])

In [11]:
x = np.array(all_x)

y = np.array(all_y).reshape(-1, 1)

In [12]:
x.shape

(411, 315360, 1)

In [14]:
X = x.squeeze()
X.shape # shape of X = (m,n )

(411, 315360)

In [15]:
y.shape

(411, 1)

In [19]:
for i in y:
    print(i[0])

00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
1

In [20]:
y_to_b = []
for i in y:
    row = [0]*14
    row[int(i[0])-1] = 1
    y_to_b.append(row)
y_to_b = np.array(y_to_b)

In [21]:
print(y_to_b.shape)
print(X.shape)

(411, 14)
(411, 315360)


In [22]:
class model():
    def __init__(self, X, num_class): 
        self.weights = np.random.rand(X.shape[1], num_class) 
        #self.weights = np.zeros((X.shape[1], num_class)) 
        self.biases = np.zeros(shape = (1, num_class))
        self.X = X
        self.num_class = num_class
         
    def forward(self, X, w, b):
        return np.matmul(X, w) + b
    
    def softmax(self, z):
        z -= np.max(z, axis = 1).reshape(-1, 1)
        return np.exp(z) / np.array(np.sum(np.exp(z), axis = 1)).reshape(-1, 1)
        
    def cost(self, softmax, y):
        #m = y.shape[0]
        #cost = (- 1 / m) * np.sum(y * np.log(softmax) + (1 - y) * (np.log(1 - softmax)))  # compute cost
        
        #loss = (y * np.log(softmax)) + (1-y) * (np.log(1 - softmax))
        loss = np.multiply(y, np.log(softmax)) + np.multiply(y, np.log(1 - softmax))
        num_image = y.shape[0]
        cost = - np.sum(loss) / num_image
        cost = np.squeeze(cost)
        return cost
    
    def gradient_cost(self, X, softmax, y):
        loss = self.loss
    #this loss1 is the cross entropy cost
    
    def cross_entropy_cost(self, softmax, y):
        cost = -np.sum(np.multiply(y, np.log(softmax + 0.01)))
        #cost = -np.sum(np.multiply(y, np.log(softmax[:, y])))
        return cost
    
         
    def backward(self, X, softmax, y):
        m = X.shape[1]
        dw = (1 / m) * np.dot(X.T, (softmax - y))
        db = (1 / m) * np.sum(softmax - y)
        return dw, db
    
    def update_parameters(self, dw, db, alpha, w_old, b_old):
        w_new = w_old - alpha * dw
        b_new = b_old - alpha * db 
        return w_new, b_new
    
    def update_parameters_2(self, X, cost, lr, w_old, b_old):
        
        w_new = w_old - np.dot(lr, np.multiply(X, cost))
        b_new = b_old - np.dot(lr, np.sum(cost))
        
        return w_new, b_new


In [23]:
num_class = 14
model = model(X, num_class)
w = model.weights
b = model.biases

In [24]:
print(w.shape)
print(w)

(315360, 14)
[[0.59438558 0.0761503  0.36539519 ... 0.14234341 0.37626372 0.0522706 ]
 [0.50397367 0.05721941 0.65289646 ... 0.55298317 0.88478932 0.73324626]
 [0.06385445 0.75558441 0.88001538 ... 0.36454949 0.14050563 0.84066262]
 ...
 [0.02513943 0.65839338 0.13813729 ... 0.2373734  0.78393018 0.23763207]
 [0.03345367 0.90525867 0.17498159 ... 0.57808999 0.5867684  0.3617925 ]
 [0.81957762 0.00952564 0.16355166 ... 0.24965257 0.11121821 0.0775684 ]]


In [25]:
print(b.shape)
print(b)

(1, 14)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# The Forward Pass

In [26]:
a = model.forward(X, w, b)

In [27]:
print("The shape of the output array becomes", a.shape)
print("\nThe forward output of all the image\n")
print(a)
#print("\nThe forward output of the 20th image how much it is means the prediction through the 14 class")
#print(a[20])

The shape of the output array becomes (411, 14)

The forward output of all the image

[[389951.50477573 394807.43941101 393362.2249713  ... 390414.58909842
  381969.73201849 391900.86681566]
 [415925.23812816 415300.58780628 411527.57418472 ... 408218.79611347
  405915.16060016 406541.62237463]
 [430073.24905149 431690.58115578 431241.69380878 ... 436884.93008754
  431699.87266732 424293.93098348]
 ...
 [570935.07041408 577974.78400585 580649.86810861 ... 562961.43455461
  568121.45368691 567856.24999438]
 [605544.66815666 617549.15867877 606016.95425982 ... 604897.64972499
  600829.68623392 611064.62834541]
 [570741.23375456 572383.76023949 573550.61883305 ... 580228.7949793
  577347.43832165 580748.4872004 ]]


## Expected Result

The shape of the output array becomes (411, 14)

The forward output of all the image

The forward output of the 20th image how much it is means the prediction through the 14 class [474244.45342753 480183.38392286 471674.62918 478515.06985859 464605.62585467 477473.93646667 473011.59488537 478671.92048778 461979.22081921 470068.8819789 480149.49769161 469849.74504296 466378.11909758 472072.12665338]

# The Softmax

In [28]:
softmax = model.softmax(a)
print("The shape of the prediction of an array output becomes", softmax.shape)
print("\n", softmax)

print("\nchecking the softmax by summing their values")
print(np.sum(softmax[165]))

The shape of the prediction of an array output becomes (411, 14)

 [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.31951606e-295
  0.00000000e+000 0.00000000e+000]
 ...
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 1.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 1.99772676e-226
  0.00000000e+000 1.00000000e+000]]

checking the softmax by summing their values
1.0


# The loss

In [29]:
#loss = model.cross_entropy_cost(softmax, y_to_b)
loss = model.cost(softmax, y_to_b)

print("the loss is", loss)


the loss is nan


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply


In [38]:
cost = model.cross_entropy_cost(softmax, y_to_b)

# Back Propagation

In [30]:
dw, db = model.backward(X, softmax, y_to_b)

In [31]:
print(dw)
print(dw.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(315360, 14)


# Update parameters


In [39]:
u_w1, u_b1 = model.update_parameters(dw, db, 0.01, w, b)
w, b = model.update_parameters_2(X, cost, 0.01, w, b)

ValueError: operands could not be broadcast together with shapes (315360,14) (411,315360) 

In [40]:
print(u_w1)
print(u_w1.shape)

[[0.59438558 0.0761503  0.36539519 ... 0.14234341 0.37626372 0.0522706 ]
 [0.50397367 0.05721941 0.65289646 ... 0.55298317 0.88478932 0.73324626]
 [0.06385445 0.75558441 0.88001538 ... 0.36454949 0.14050563 0.84066262]
 ...
 [0.02513943 0.65839338 0.13813729 ... 0.2373734  0.78393018 0.23763207]
 [0.03345367 0.90525867 0.17498159 ... 0.57808999 0.5867684  0.3617925 ]
 [0.81957762 0.00952564 0.16355166 ... 0.24965257 0.11121821 0.0775684 ]]
(315360, 14)


## The second forward pass

In [42]:
forward2 = model.forward(X, u_w1, u_b1)

softmax2 = model.softmax(forward2)

In [45]:
loss2 = model.cross_entropy_cost(softmax2, y_to_b)


In [46]:
print(loss2)

1740.4259693849951


# For Loops

In [49]:
num_class = 14
model = model(X, num_class)

TypeError: 'model' object is not callable

In [50]:
w = model.weights
b = model.biases

In [51]:
#print("initial weight\n\t", w)
#print("\ninitial bias\n\t", b )
for i in range(10):
    #print("\nweight ", i , "\n\t", w)
    #print("\nbias ", i, "\n\t", b )
    forward = model.forward(X, w, b)
    soft = model.softmax(forward)
    #cost = model.cross_entropy_cost(soft, y_to_b)
    cost = model.cost(soft, y_to_b)
    print("The Loss in Step ", i , " is \t", cost)
    dw, db = model.backward(X, soft, y_to_b)
    w, b = model.update_parameters(dw, db, 0.001, w, b)
    #w, b = model.update_parameters_2(X, cost, 0.01, w, b)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply


The Loss in Step  0  is 	 nan
The Loss in Step  1  is 	 nan
The Loss in Step  2  is 	 nan
The Loss in Step  3  is 	 nan
The Loss in Step  4  is 	 nan
The Loss in Step  5  is 	 nan
The Loss in Step  6  is 	 nan
The Loss in Step  7  is 	 nan
The Loss in Step  8  is 	 nan
The Loss in Step  9  is 	 nan


In [54]:
import tensorflow as tf
import numpy as np

In [55]:
XX = tf.placeholder(tf.float32, shape = (411, 720*438*1))

In [56]:
yy = tf.placeholder(tf.float32, shape=[411, 14])

In [57]:
ww = tf.Variable(tf.zeros((720*438*1, 14)))

In [58]:
bb = tf.Variable(tf.zeros((1, 14)))

In [59]:
print(X.shape)
print(XX.shape)

(411, 315360)
(411, 315360)


In [60]:
print(ww.shape)

print(bb.shape)

(315360, 14)
(1, 14)


In [61]:
print(yy.shape)

(411, 14)


In [62]:
mat = tf.matmul(XX, ww)
logit = tf.add(mat, bb)

In [63]:
logit.shape

TensorShape([Dimension(411), Dimension(14)])

In [64]:
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=yy, logits=logit))

In [65]:
train = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [66]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(15):
        losss,_ = sess.run([loss,train], feed_dict={XX:X, yy:y_to_b})
        print(str(losss))

1084.6531
255267.05
1000963.75
3412283.8
343078.34
3390.4219
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [67]:
def tf_loss(logits, labels):
    cross_entropy_per_number = tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels, logits = logits)
    cross_entropy = tf.reduce_mean(cross_entropy_per_number)
    tf.add_to_collection("loss", cross_entropy)
    return cross_entropy


In [72]:
print(tf_loss(logit,

SyntaxError: unexpected EOF while parsing (<ipython-input-72-e37a5ce7cbd9>, line 1)

In [73]:
def evaluation(logits, labels):
    prediction = tf.argmax(logits, 2)
    actual = tf.argmax(labels, 2)
    equal = tf.equal(prediction, actual)
    # equal = tf.reduce_all(equal, 1)
    accuracy = tf.reduce_mean(tf.cast(equal, tf.float32), name="accuracy")
    return accuracy


In [74]:
def train(loss, learning_rate=0.00001):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    print(train_op)
    return train_op

In [75]:
train(tf_loss(logit, yy))

name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_Variable/ApplyGradientDescent"
input: "^GradientDescent_1/update_Variable_1/ApplyGradientDescent"



<tf.Operation 'GradientDescent_1' type=NoOp>